In [1]:
include("./src/methods.jl")
using PyPlot

In [2]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,varsplit = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_simplex = x_simplex[:,2]
simplex_to_circus = variables_relation(varsplit,3)
x_circus = variable_simplex_to_circus(simplex_to_circus,x_simplex)

3-element Array{Float64,1}:
  4.0
  0.0
 -4.0

In [3]:
x_simplex = variable_circus_to_simplex(simplex_to_circus,x_circus,cs,As,bs)

8-element Array{Float64,1}:
 4.0
 0.0
 0.0
 0.0
 4.0
 0.0
 0.0
 0.0

#### Phase 1


In [4]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

x = [0,0,0,0,0,0,0,4,1]
cs,As,bs,varsplit = simplex_format(c,A,b)

([0, 0, 0, 1, -1, 0, 0, 0, 0], [-1.0 1.0 … 0.0 0.0; -1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; -1.0 0.0 … 0.0 1.0], [0.0; 0.0; 4.0; -1.0], [2, 3])

In [5]:
cpi, Api, bpi, xpi,bfs_pi,nfs_pi = phaseI_simplex_problem(As,bs)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0], [-1.0 1.0 … 0.0 0.0; -1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; 1.0 -0.0 … 0.0 1.0], [0.0; 0.0; 4.0; 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 1.0], [10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
x_simplex = SimplexFromBFS(cpi,Api,bpi,xpi,index_bfs=bfs_pi,index_nfs=nfs_pi)

x_simplex = x_simplex[1:9,2]

simplex_to_circus = variables_relation(varsplit,3)
x_circus = variable_simplex_to_circus(simplex_to_circus,x_simplex)

3-element Array{Float64,1}:
  1.0
  0.0
 -1.0

In [21]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
xo = x_circus

pygui(true)
px= [1,4 ,4 ,1 ,1,1,1 ,1,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [-1,-4,0 ,3 ,-1,3,3 ,3,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([xo[1]],[xo[2]],[xo[3]])

PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x7f3164e83eb0>